In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
img_size = 128
IMAGE_SIZE = [img_size, img_size]

In [ ]:
'''train_path="/home/nayan/Documents/Thesis/dataset/test_trn jonathanrjpereira rice-disease/train_set"
test_path="/home/nayan/Documents/Thesis/dataset/test_trn jonathanrjpereira rice-disease/test_set"
val_path="/home/nayan/Documents/Thesis/dataset/test_trn jonathanrjpereira rice-disease/test_set"'''

train_path="/home/nayan/Documents/Thesis/dataset/new clean/train"
test_path= "/home/nayan/Documents/Thesis/dataset/new clean/test"
val_path=  "/home/nayan/Documents/Thesis/dataset/new clean/test"

In [ ]:
x_train=[]

for folder in os.listdir(train_path):
    sub_path=train_path+"/"+folder
    print(sub_path)
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(img_size,img_size))
        x_train.append(img_arr)

In [ ]:
x_test=[]

for folder in os.listdir(test_path):
    sub_path=test_path+"/"+folder
    print(sub_path)
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(img_size,img_size))
        x_test.append(img_arr)

In [ ]:
x_val=[]

for folder in os.listdir(val_path):
    sub_path=val_path+"/"+folder
    print(sub_path)
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(img_size,img_size))
        x_val.append(img_arr)

In [ ]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

In [21]:
train_x.shape,test_x.shape,val_x.shape


((441, 128, 128, 3), (76, 128, 128, 3), (76, 128, 128, 3))

In [22]:
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [24]:
# train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True)

train_datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None)
test_datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None)
val_datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None)


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (img_size, img_size),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (img_size, img_size),
                                            batch_size = 32,
                                            class_mode = 'sparse')

val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (img_size, img_size),
                                            batch_size = 32,
                                            class_mode = 'sparse')

Found 441 images belonging to 3 classes.
Found 76 images belonging to 3 classes.
Found 76 images belonging to 3 classes.


In [25]:
training_set.class_indices


{'Brown_Spot': 0, 'Hispa': 1, 'Leaf_Blast': 2}

In [26]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes
train_y.shape,test_y.shape,val_y.shape

((441,), (76,), (76,))

In [27]:
# add preprocessing layer to the front of VGG
resnet = ResNet101V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [28]:
for layer in resnet.layers:
    layer.trainable = False

In [29]:
x = Flatten()(resnet.output)

prediction = Dense(3, activation='softmax')(x)

In [30]:
# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 66, 66, 64)   0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [31]:
# tell the model what cost and optimization method to use
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [33]:
# fit the model
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=30,  
  batch_size=32,shuffle=True)

Epoch 1/30
14/14 [==============================] - 5s 346ms/step - loss: 0.5047 - accuracy: 0.7982 - val_loss: 0.7751 - val_accuracy: 0.6711
Epoch 2/30
14/14 [==============================] - 5s 355ms/step - loss: 0.4905 - accuracy: 0.8050 - val_loss: 0.7423 - val_accuracy: 0.7105
Epoch 3/30
14/14 [==============================] - 5s 357ms/step - loss: 0.4894 - accuracy: 0.8095 - val_loss: 0.7608 - val_accuracy: 0.6974
Epoch 4/30
14/14 [==============================] - 5s 372ms/step - loss: 0.4844 - accuracy: 0.8277 - val_loss: 0.7723 - val_accuracy: 0.6579
Epoch 5/30
14/14 [==============================] - 5s 360ms/step - loss: 0.4723 - accuracy: 0.8005 - val_loss: 0.7596 - val_accuracy: 0.6842
Epoch 6/30
14/14 [==============================] - 5s 362ms/step - loss: 0.4623 - accuracy: 0.8277 - val_loss: 0.7540 - val_accuracy: 0.6842
Epoch 7/30
14/14 [==============================] - 5s 372ms/step - loss: 0.4543 - accuracy: 0.8095 - val_loss: 0.7590 - val_accuracy: 0.6842
Epoch 